# Démonstration Spark & Hive Metastore

In [1]:
import os
from IPython.display import Image
from IPython.core.display import HTML

from pyspark.sql import SparkSession

In [7]:
Image(url="https://spark.apache.org/docs/latest/img/cluster-overview.png")

In [2]:
spark = (SparkSession 
         .builder
         # configuration de la dynamicAllocation
         # .config("spark.dynamicAllocation.enabled","true")
         # .config("spark.dynamicAllocation.initialExecutors","1")
         # .config("spark.dynamicAllocation.minExecutors","1")
         # .config("spark.dynamicAllocation.maxExecutors","10")
         .config("spark.dynamicAllocation.executorIdleTimeout", "20")
         .enableHiveSupport()
         .getOrCreate()
        )

sc = spark.sparkContext

2023-04-24 13:24:01,320 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-04-24 13:24:04,181 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
2023-04-24 13:24:04,660 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [3]:
!kubectl get pods -l spark-role=executor

NAME                                    READY   STATUS    RESTARTS   AGE
pyspark-shell-5f97ba87b36f849a-exec-1   1/1     Running   0          6s


## CSV to Parquet

__Partie illustrative, non reproductible__

In [10]:
!mc head -n 2 s3/projet-onyxia/demo/rp/individus.csv

region_residence;departement_residence;commune_residence;region_travail;departement_travail;commune_travail;commune_anterieure;commune_etude;pays_naissance;poids;sexe;statut_pro;densite;recherche_emploi;diplome;age;variable00;variable01;variable02;variable03;variable04;variable05;variable06;variable07;variable08;variable09;variable10;variable11;variable12;variable13;variable14;variable15;variable16;variable17;variable18;variable19;variable20;variable21;variable22;variable23;variable24;variable25;variable26;variable27;variable28;variable29;variable30;variable31;variable32;variable33;variable34;variable35;variable36;variable37;variable38;variable39;variable40;variable41;variable42;variable43;variable44;variable45;variable46;variable47;variable48;variable49
53;22;22170;53;22;22170;22170;22170;99;1.197604824388689;2;Z;3;0;2;3;930660000002340;0001;006;9316;652242.5;6871102.2000000002;2;2017;01;1989;05;09;028027;027;027;2;75113;999;01;11;2;2;111;111;1;ZZZ;997;00;997;01;2;2;Z;YYYYY;ZZZZZ;2;93

In [12]:
(spark.read
      .format("csv")
      .options(header='true', inferschema='true', delimiter=';')
      .load("s3a://projet-onyxia/demo/rp/individus.csv")
      .write
      .mode('overwrite')
      .parquet("s3a://projet-onyxia/demo/rp/individus.parquet")
     )

2022-04-28 09:18:56,481 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [16]:
!mc ls s3/projet-onyxia/demo/rp/individus.parquet | head -n 5

[2022-04-28 09:21:44 UTC]     0B _SUCCESS
[2022-04-28 09:21:11 UTC]  10MiB part-00000-033b363c-c084-4e78-8487-496b6d8e8bd3-c000.snappy.parquet
[2022-04-28 09:21:23 UTC]  10MiB part-00001-033b363c-c084-4e78-8487-496b6d8e8bd3-c000.snappy.parquet
[2022-04-28 09:20:47 UTC]  10MiB part-00002-033b363c-c084-4e78-8487-496b6d8e8bd3-c000.snappy.parquet
[2022-04-28 09:21:35 UTC]  10MiB part-00003-033b363c-c084-4e78-8487-496b6d8e8bd3-c000.snappy.parquet


## Requêtes SQL

In [7]:
df_parquet = (
    spark.read.format("csv")
    .options(header='true', inferschema='true', delimiter=';').
    load("s3a://projet-formation/diffusion/bonnes-pratiques-r/rp_2016_individu_sample.csv")
)
df_parquet.createOrReplaceTempView("individus")

2023-04-24 13:27:24,041 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [12]:
spark.sql("""
SELECT *
FROM individus
WHERE region='52'
""").show(2)

+------+-------+-----+----+-----+----+------+------+-------+----+-----+----+-----+----+------+-----+----+----+----+-----+----+----+----+----+----+------+---+---+---+----+----+-----+-------+---+-----+----+----+---------+----+----+----+------+------+---+-----+----+----+----+-----+-----+------+----------------+----+------+-----+----+----+--------+----+----+----+----+-----+-----+-----+-----+----+------+------+-----+----+-----+----+-----+------+----+----+----+-------+----+---+----+---------+-----+----+----+-------+---+-----+----+-----+----+-----+-----+---+----+---+
|region|  nummr|achlr|aemm|aemmr|aged|ager20|agerev|agerevq|anai|anarr|anem|anemr|apaf|arrivr|ascen|bain|bati|catl|catpc|chau|chfl|chos|clim|cmbl|couple|cs1|cs2|cs3|cuis|dept|derou|dipl_15|eau|egoul|elec|empl|     epci|etud|garl|hlml|iletud|iletuu|ilt|iltuu|immi|inai|inat|infam|inper|inperf|          ipondi|iran|iranuu|lienf|lprf|lprm|metrodom|moco|modv|na38|na88|naf08|naidt|nat13|nat49|natc|natn12|natn49|natnc|nbpi|nperr|numf|or

## Partage des données via un Hive Metastore

__TO BE UPDATED__

In [10]:
spark.sql('show tables;').show()

2023-04-24 13:28:51,572 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |individus|       true|
+---------+---------+-----------+



In [4]:
schema_str = ', '.join([' '.join(x) for x in df_parquet.dtypes])

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS individus_rp
({schema_str})
STORED as parquet LOCATION 'projet-onyxia/demo/rp/individus.parquet'
""")

2022-04-27 11:44:06,870 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [5]:
spark.sql('show tables;').show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|  default|individus_part_re...|      false|
|  default|     individusregion|      false|
+---------+--------------------+-----------+



In [4]:
df_req1 = spark.sql("""
SELECT sum(poids), sexe, densite, statut_pro
FROM individus_part_region_residence
WHERE region_residence='44' AND region_travail != '44' AND statut_pro != 'Z'
GROUP BY sexe, densite, statut_pro
""").show()

+------------------+----+-------+----------+
|        sum(poids)|sexe|densite|statut_pro|
+------------------+----+-------+----------+
| 84860.02165580631|   1|      4|         1|
|  81647.7300888066|   1|      3|         1|
| 20318.62707002139|   1|      3|         2|
| 4594.674335848755|   2|      2|         2|
|21197.941394322574|   2|      4|         2|
| 81280.18280711232|   2|      3|         1|
| 85804.86044169613|   2|      4|         1|
| 2422.462061829637|   1|      1|         1|
|20696.859938120786|   2|      3|         2|
|21014.734028133804|   1|      4|         2|
| 18598.93587046766|   1|      2|         1|
|2403.5120489857827|   2|      1|         1|
|18421.386742139224|   2|      2|         1|
|4560.4938174609615|   1|      2|         2|
| 607.7579233084527|   1|      1|         2|
| 592.3437398684353|   2|      1|         2|
+------------------+----+-------+----------+



In [5]:
df_req2 = spark.sql("""
SELECT sum(poids), age, diplome
FROM individus_part_region_residence
WHERE recherche_emploi IN ('1','2') AND region_residence='11' AND commune_residence IN ('75013','75014','75015','92100','92130','92075','92240','92120','94250','94200')
GROUP BY diplome, age
""").show()

+------------------+---+-------+
|        sum(poids)|age|diplome|
+------------------+---+-------+
| 9.177751090295155|  0|      1|
| 6.360279128629814|  1|      6|
|11.422012968672975|  1|      3|
|14.023248192593126|  4|      7|
|17.155410381367968|  2|      2|
|10.180609173643692|  7|      1|
|12.058477661296191|  1|      7|
| 9.255402890330407|  3|      2|
|11.077211876128839|  2|      1|
| 7.591013552404388|  7|      0|
|17.704675201373025|  0|      6|
|11.098099052856005|  9|      1|
|6.5043199507982346|  5|      0|
|11.063367224180798|  7|      5|
|10.859354234442353|  8|      6|
| 8.077992082811662|  1|      1|
|10.731451889825916|  5|      6|
| 7.404974459515381|  9|      4|
|10.519514716965539|  5|      3|
|16.322972771017838|  5|      2|
+------------------+---+-------+
only showing top 20 rows



In [6]:
df_req3 = spark.sql("""
SELECT sum(poids), pays_naissance, departement_residence
FROM individus_part_region_residence
WHERE region_residence='44'
GROUP BY pays_naissance, departement_residence
""").show()

+------------------+--------------+---------------------+
|        sum(poids)|pays_naissance|departement_residence|
+------------------+--------------+---------------------+
|  86885.8063479457|            51|                   52|
| 74511.08026494365|            21|                   68|
|101801.91621098612|            41|                   55|
| 88498.68533646171|            12|                   10|
| 86962.65446028857|            31|                   52|
|148134.84723748424|            12|                   57|
| 87066.47522814616|            21|                   52|
|105343.26853180646|            31|                   67|
| 74258.88213580017|            31|                   68|
|103167.34376872572|            51|                   88|
| 103272.6317022982|            31|                   88|
|104006.69954490622|            21|                   88|
|101665.45053942443|            10|                   55|
|125388.10926533854|            12|                   51|
| 92142.580919